In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob

In [2]:
# Load a csv file
df = pd.read_csv('selected_songs - subsetNew.csv')
# Remove rows wirh Check = N
df = df[df['Check'] == 'Y']
# Reset the index
df = df.reset_index(drop=True)
df

,Index,Artist,Title,Mood,Check
0,ML100,Bad Company,Can't Get Enough,happy,Y
1,ML1003,Los Lobos,Come On Let's Go,happy,Y
2,ML1008,Hezekiah Walker & The Love Fellowship Crusade ...,Power Belongs To God,happy,Y
3,ML1009,Donald Byrd,Think Twice,happy,Y
4,ML1011,Inner City,Good Life,happy,Y
...,...,...,...,...,...
168,ML1203,Bettye LaVette,It Don't Come Easy,relaxed,Y
169,ML1207,Johnny O.,Fantasy Girl,relaxed,Y
170,ML1210,Harlem,Spray Paint,relaxed,Y
171,ML1211,Fleet Foxes,Sun It Rises,relaxed,Y


In [3]:
# Create a folder named subset80verified if it doesn't exist
import os
if not os.path.exists('subsetNew'):
    os.makedirs('subsetNew')

In [ ]:
# The df dataframe has a Mood column which corresponds to the folder name in selected_lyrics folder under which the lyrics are stored with file named as df['Index]+ something
# Get these songs and put thme in the subset80verified folder
import shutil
import glob
for index, row in df.iterrows():
    # Find files with the specified pattern
    files_to_copy = glob.glob(f"../../temp/selected_lyrics/{row['Mood']}/{row['Index']}_*.txt")
    print(files_to_copy)
    # Copy each matching file to 'subset80verified'
    for file_to_copy in files_to_copy:
        shutil.copy(file_to_copy, 'subsetNew')

In [6]:
# Print number of files in the subset80verified folder
print(len(os.listdir('subsetNew')))

173


In [7]:
# Add a new column to the df dataframe which contains the filename of the lyrics file
df['Filename'] = ''
for index, row in df.iterrows():
    # Find files with the specified pattern
    files_to_copy = glob.glob(f"../OpenAI GPT3/selected_lyrics/{row['Mood']}/{row['Index']}_*.txt")
    # Copy each matching file to 'subset80verified'
    for file_to_copy in files_to_copy:
        df.at[index, 'Filename'] = file_to_copy.split('/')[-1]

In [ ]:
df

In [9]:
# system_content = "You are a chatbot that, when prompted with song lyrics, predicts one of the emotions ('Happy', 'Sad', 'Angry', or 'Relaxed') without providing any explanation. Reply with only the emotion name."
system_content = "You are a chatbot that, when prompted with song lyrics, predicts one of the emotions ('Happy', 'Sad', 'Angry', or 'Relaxed')\
    without providing any explanation. Reply with only the emotion name. You do not retain any previous information regarding the lyrics given to you. \
        You specialize in analysing the given song lyrics and predicting the emotion of the song."

In [10]:
# We need to create a new array of the following format
# {"messages": [{"role": "system", "content": system_content}, {"role": "user", "content": <content in file based on filename>}, {"role": "assistant", "content": <corresponding label in Mood column of df>}]}
prompt_array = []
count = 0
for index, row in df.iterrows():
    count += 1
    # Open the file and read the contents
    with open(f"subsetNew/{row['Filename']}", 'r') as f:
        content = f.read()
    # Shift content in first line after 'lyrics' word to second line
    content = content.replace('Lyrics', 'Lyrics\n')
    # Remove the first line from the content
    content = content.split('\n', 1)[1]
    # if count == 2:
    #     print(content)
    #     break
    # # Create the prompt
    prompt = {"messages": [{"role": "system", "content": system_content}, {"role": "user", "content": content}, {"role": "assistant", "content": row['Mood']}]}
    # print(prompt, row['Index'])
    # # Append the prompt to the prompt_array
    prompt_array.append(prompt)

In [11]:
# In prmompt array, find if any key has a value "Jesse" in the content prompt['messages'][1]['content']
for prompt in prompt_array:
    print(prompt['messages'][1]['content'], prompt['messages'][2]['content'])
    if prompt['messages'][2]['content'] == "Jesse":
        print(prompt)

[Intro]

[Verse 1]
Well, I take whatever I want
And, baby, I want you
You give me something I need
Now tell me I got something for you

[Pre-Chorus]
Come on
Come on
Come on and do it
Come on and-uh do what you do

[Chorus]
I can't get enough of your love
I can't get enough of your love
I can't get enough of your love
Listen

[Verse 2]
Well, it's late, and I want love
Love that's gonna break me in two
Don't you hang me up in your doorway
Don't you hang me up like you do, say it!
You might also like[Pre-Chorus]
Come on
Come on
Come on and do it
Come on, come on, do what you do, Woo!

[Chorus]
I can't get enough of your love
I can't get enough of your love
I can't get enough of your love

[Bridge]
Well, I say now!

[Outro]
I can't get enough of your love
I can't get enough of your love
I can't get enough of your love
Say it again, aah
I can't get enough of your love
I can't get enough of your love
I can't get enough of your love
I've got to tell ya baby, that I
I love you so much I can't 

In [ ]:
# Split the prompt_array into train, validation, test splits (58:)
from sklearn.model_selection import train_test_split
train, test = train_test_split(prompt_array, test_size=0.42, random_state=42)
train, val = train_test_split(train, test_size=0.2, random_state=42)

# Print the number of Mood labels in train and test sets identifyeid by thhe third dictionary's second key
print(len(train),  len(val),len(test),)
print(train[0]['messages'][2]['content'], test[0]['messages'][2]['content'], val[0]['messages'][2]['content'])
# Count total number of each Mood label in train and test sets
train_count = {}
test_count = {}
val_count = {}
for prompt in train:
    if prompt['messages'][2]['content'] in train_count:
        train_count[prompt['messages'][2]['content']] += 1
    else:
        train_count[prompt['messages'][2]['content']] = 1
for prompt in test:
    if prompt['messages'][2]['content'] in test_count:
        test_count[prompt['messages'][2]['content']] += 1
    else:
        test_count[prompt['messages'][2]['content']] = 1
for prompt in val:
    if prompt['messages'][2]['content'] in val_count:
        val_count[prompt['messages'][2]['content']] += 1
    else:
        val_count[prompt['messages'][2]['content']] = 1
print(train_count, test_count, val_count,sep='\n')

In [13]:
# Save the train and test sets as jsonl files
import jsonlines
with jsonlines.open('train_promptV2.jsonl', 'w') as f:
    f.write_all(train)
with jsonlines.open('test_promptV2.jsonl', 'w') as f:
    f.write_all(test)
with jsonlines.open('val_promptV2.jsonl', 'w') as f:
    f.write_all(val)
# Store prompt_array
with jsonlines.open('prompt_array_promptV2.jsonl', 'w') as f:
    f.write_all(prompt_array)

In [14]:
# !pip3 install tiktoken
import json
import tiktoken
encoding = tiktoken.encoding_for_model('gpt-3.5-turbo') 
encoding.encode("tiktoken is great!")

[83, 1609, 5963, 374, 2294, 0]

In [15]:
# For each second dictionary's second ket in the prompt_array, encode it using tiktoken and find the length of it and store in a array
lengths = []
temp = []
for prompt in prompt_array:
    lengths.append(len(encoding.encode(prompt['messages'][1]['content'])))
    temp.append([len(encoding.encode(prompt['messages'][1]['content'])), prompt['messages'][1]['content'], prompt['messages'][2]['content']])
# Sort the lengths array descendingly
lengths.sort(reverse=True)
lengths

[1186,
 1042,
 932,
 825,
 773,
 743,
 740,
 715,
 689,
 682,
 664,
 646,
 642,
 639,
 622,
 596,
 585,
 571,
 560,
 547,
 545,
 542,
 534,
 533,
 532,
 527,
 527,
 511,
 498,
 486,
 481,
 478,
 475,
 473,
 469,
 465,
 455,
 454,
 447,
 443,
 439,
 431,
 425,
 416,
 409,
 406,
 394,
 391,
 382,
 375,
 374,
 372,
 369,
 368,
 366,
 359,
 358,
 357,
 355,
 353,
 351,
 351,
 350,
 349,
 343,
 336,
 336,
 335,
 334,
 332,
 331,
 330,
 329,
 328,
 325,
 320,
 319,
 318,
 316,
 313,
 313,
 312,
 311,
 309,
 305,
 303,
 301,
 298,
 297,
 290,
 286,
 286,
 280,
 277,
 277,
 275,
 271,
 269,
 268,
 268,
 265,
 263,
 263,
 260,
 259,
 258,
 257,
 256,
 255,
 254,
 253,
 251,
 248,
 247,
 246,
 243,
 243,
 241,
 240,
 239,
 238,
 230,
 230,
 229,
 225,
 224,
 223,
 221,
 220,
 220,
 214,
 211,
 207,
 207,
 203,
 203,
 202,
 199,
 198,
 194,
 192,
 191,
 191,
 184,
 180,
 179,
 177,
 176,
 173,
 170,
 167,
 165,
 165,
 164,
 162,
 162,
 162,
 158,
 152,
 147,
 147,
 143,
 143,
 143,
 132,
 131,
 1

In [16]:
temp.sort(reverse=True)
temp

[[1186,
  "\n[Intro: Rocko & Gucci Mane]\nI love things about her\nI love the way she treat me\nI love the way she fuck me\nI love the way she suck me\nI love things about her\nI love the way she treat me\nI love the way she fuck me (It's Gucci)\nI love the way she suck me (It's Gucci goin' in), ayy (Go)\n\n[Verse 1: Gucci Mane]\nI'm a trapaholic, all I do is ball\nTear the mall down now, trick, don't flag me down (It's Gucci)\nSee me downtown, 'Rari (Skrrt), I didn't mean to shit on you\nExcuse me, sorry, verse so slick, make my video marvelous\nPolo to the floor, Ralph'd down like Carlton\nTry the CEO, they gon' put you on a carton (Carton)\nI didn't mean no harm, but my diamonds are dumb (Bling, bling)\nPlenty of rocks on my arm (Brr, brr)\nMe and Rocko the Don, East Atlanta the Slum\nWhere I'm from, we got a gun, in the club havin' fun (It's Gucci)\nAnd after it's done, no more coaches to call it (Yeah)\nAfter party, bachelor party, they shoulda called it (It's Gucci)\n\n[Chorus: R

In [17]:
# Add them all up
sum(lengths)

58684

In [18]:
import json
import tiktoken # for token counting
import numpy as np
from collections import defaultdict

In [19]:
data_path = 'train_173_promptV2.jsonl'
# Load the dataset
with open(data_path, 'r', encoding='utf-8') as f:
    dataset = [json.loads(line) for line in f]
# Initial dataset stats
print("Num examples:", len(dataset))
print("First example:")
for message in dataset[0]['messages']:
    print(message)

Num examples: 80
First example:
{'role': 'system', 'content': "You are a chatbot that, when prompted with song lyrics, predicts one of the emotions ('Happy', 'Sad', 'Angry', or 'Relaxed')    without providing any explanation. Reply with only the emotion name. You do not retain any previous information regarding the lyrics given to you.         You specialize in analysing the given song lyrics and predicting the emotion of the song."}
{'role': 'user', 'content': '\n[Intro]\nAh Yeah\nTime to come back home y`all\nCome on, come on, come on\n\n[Verse 1]\nFor God I`m a slave To get to heaven Reverend said I gotta be saved\nAnd behave And also stuck in my ways Give Him praise\nWords can`t express what I feel inside of my flesh\nEvery breath is giving Him glory until my death (I feel blessed)\nCuz everyday I`m seeing Him work (I feel blessed)\nI don`t wanna do no more dirt (I feel blessed)\nIf the path I was on was wrong (I feel blessed)\nIt`s time to come home\n\n[Verse 2]\nIt doesn`t matter

In [ ]:
# Format error checks
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue
        
    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue
        
    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1
        
        if any(k not in ("role", "content", "name", "function_call") for k in message):
            format_errors["message_unrecognized_key"] += 1
        
        if message.get("role", None) not in ("system", "user", "assistant", "function"):
            format_errors["unrecognized_role"] += 1
            
        content = message.get("content", None)
        function_call = message.get("function_call", None)
        
        if (not content and not function_call) or not isinstance(content, str):
            format_errors["missing_content"] += 1
    
    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

In [21]:
# not exact!
# simplified from https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")

In [22]:
# Warnings and tokens counts
n_missing_system = 0
n_missing_user = 0
n_messages = []
convo_lens = []
assistant_message_lens = []

for ex in dataset:
    messages = ex["messages"]
    if not any(message["role"] == "system" for message in messages):
        n_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        n_missing_user += 1
    n_messages.append(len(messages))
    convo_lens.append(num_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))
    
print("Num examples missing system message:", n_missing_system)
print("Num examples missing user message:", n_missing_user)
print_distribution(n_messages, "num_messages_per_example")
print_distribution(convo_lens, "num_total_tokens_per_example")
print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")
n_too_long = sum(l > 4096 for l in convo_lens)
print(f"\n{n_too_long} examples may be over the 4096 token limit, they will be truncated during fine-tuning")

Num examples missing system message: 0
Num examples missing user message: 0

#### Distribution of num_messages_per_example:
min / max: 3, 3
mean / median: 3.0, 3.0
p5 / p95: 3.0, 3.0

#### Distribution of num_total_tokens_per_example:
min / max: 149, 1280
mean / median: 439.65, 389.5
p5 / p95: 257.0, 697.8000000000006

#### Distribution of num_assistant_tokens_per_example:
min / max: 1, 2
mean / median: 1.6125, 2.0
p5 / p95: 1.0, 2.0

0 examples may be over the 4096 token limit, they will be truncated during fine-tuning


In [23]:
# Pricing and default n_epochs estimate
MAX_TOKENS_PER_EXAMPLE = 4096

TARGET_EPOCHS = 3
MIN_TARGET_EXAMPLES = 100
MAX_TARGET_EXAMPLES = 25000
MIN_DEFAULT_EPOCHS = 1
MAX_DEFAULT_EPOCHS = 25

n_epochs = TARGET_EPOCHS
n_train_examples = len(dataset)
if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
    n_epochs = min(MAX_DEFAULT_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_DEFAULT_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)
print(f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training")
print(f"By default, you'll train for {n_epochs} epochs on this dataset")
print(f"By default, you'll be charged for ~{n_epochs * n_billing_tokens_in_dataset} tokens")

Dataset has ~35172 tokens that will be charged for during training
By default, you'll train for 3 epochs on this dataset
By default, you'll be charged for ~105516 tokens


In [31]:
from openai import OpenAI
api_key = "" # Add your API key here
client = OpenAI(api_key="")
client.files.create(
  file=open("train_promptV2.jsonl", "rb"),
  purpose="fine-tune"
)

FileObject(id='file-arftnUBYfuEsPjocJfnNiwSh', bytes=141261, created_at=1701293044, filename='train_173_promptV2.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [32]:

client.files.create(
  file=open("val_promptV2.jsonl", "rb"),
  purpose="fine-tune"
)

FileObject(id='file-VLPOEmu24IA8TVuWegAw8rmk', bytes=32598, created_at=1701293058, filename='val_173_promptV2.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [33]:
client.fine_tuning.jobs.create(
  training_file="file-arftnUBYfuEsPjocJfnNiwSh", 
  validation_file="file-VLPOEmu24IA8TVuWegAw8rmk",
  model="gpt-3.5-turbo-1106"
)

FineTuningJob(id='ftjob-RyLkt89mVqQtRfqGsZPYLCUU', created_at=1701293081, error=None, fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-1106', object='fine_tuning.job', organization_id='org-l1IX9f1ewHtHXjugT8SmntV4', result_files=[], status='validating_files', trained_tokens=None, training_file='file-arftnUBYfuEsPjocJfnNiwSh', validation_file='file-VLPOEmu24IA8TVuWegAw8rmk')

In [34]:
# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-RyLkt89mVqQtRfqGsZPYLCUU', created_at=1701293081, error=None, fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-1106', object='fine_tuning.job', organization_id='org-l1IX9f1ewHtHXjugT8SmntV4', result_files=[], status='validating_files', trained_tokens=None, training_file='file-arftnUBYfuEsPjocJfnNiwSh', validation_file='file-VLPOEmu24IA8TVuWegAw8rmk'), FineTuningJob(id='ftjob-OdwSsO4rVRcYqvaUpmpcQtX4', created_at=1701292920, error=Error(code='exceeded_quota', message='Creating this fine-tuning job would exceed your hard limit, please check your plan and billing details', param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-1106', object='fine_tuning.job', organization_id='org-l1IX9f1ewHtHXjugT8SmntV4', resul

In [29]:
client.fine_tuning.jobs.retrieve("ftjob-OdwSsO4rVRcYqvaUpmpcQtX4")

FineTuningJob(id='ftjob-OdwSsO4rVRcYqvaUpmpcQtX4', created_at=1701292920, error=None, fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-1106', object='fine_tuning.job', organization_id='org-l1IX9f1ewHtHXjugT8SmntV4', result_files=[], status='validating_files', trained_tokens=None, training_file='file-jMXsAKyi2UtLD8uEycQ8jQtg', validation_file='file-Ss693Xc5HwUJary3pZawLpn8')

In [48]:
# Retrieve the state of a fine-tune
client.files.content("file-6HAnsmOq8UKWNsxee8QSjRX1")

In [ ]:
!pip install evals

In [115]:
# Evaluate the fine-tuning job 
# from evals import evals
# evals.evaluate(
#     model="ft:gpt-3.5-turbo-1106:personal::8OVFs1AY",
#     fine_tuning_job="ftjob-3Gc34difuhJYLohcHzSctngm",
#     test_file="file-Ot2TAj54WJ765UZBhCPzYZK7",
#     log_level="info",
#     output_dir="evals_output"
# )

In [30]:
print(test[0]['messages'][1]['content'])


[Verse 1]
If I could make a wish
I think I'd pass
Can't think of anythin' I need
No cigarettes, no sleep, no light, no sound
Nothing to eat, no books to read

[Verse 2]
Making love with you
Has left me peaceful, warm, and tired
What more could I ask
There's nothing left to be desired
Peace came upon me and it leaves me weak
So sleep, silent angel
Go to sleep

[Chorus]
Sometimes, all I need is the air that I breathe
And to love you
All I need is the air that I breathe
Yes, to love you
All I need is the air that I breathe

[Instrumental Interlude]

[Verse 3]
Peace came upon me
And it leaves me weak
So sleep, silent angel
Go to sleep

[Chorus]
Sometimes, all I need is the air that I breathe
And to love you
All I need is the air that I breathe
Yes, to love you
All I need is the air that I breathe

[Outro]
Sometimes, all I need is the air that I breathe
And to love you
All I need is the air that I breathe
Yes, to love you
All I need is the air that I breathe
And to love you
All I need is t

In [50]:
len(test)

73

In [35]:
fine_tuned_model = "ft:gpt-3.5-turbo-1106:personal::8QMX2BHf"
# Get new_prompt as test[0]['messages'][1]['content']
output = []
for i in range(len(test)):
    new_prompt = test[i]['messages'][1]['content']
    answer = client.chat.completions.create(
      model=fine_tuned_model,
      messages=[
          {"role": "system", "content": "You are a chatbot that when prompted with song lyrics, predict one of the emotions ('Happy', 'Sad', 'Angry', or 'Relaxed') without providing any explanation. Reply with only the emotion name."},
          {"role": "user", "content": new_prompt}
      ]
    )
    print(answer.choices[0].message, test[i]['messages'][2]['content'])
    output.append([answer.choices[0].message, test[i]['messages'][2]['content']])

ChatCompletionMessage(content='relaxed', role='assistant', function_call=None, tool_calls=None) relaxed
ChatCompletionMessage(content='relaxed', role='assistant', function_call=None, tool_calls=None) happy
ChatCompletionMessage(content='angry', role='assistant', function_call=None, tool_calls=None) angry
ChatCompletionMessage(content='sad', role='assistant', function_call=None, tool_calls=None) sad
ChatCompletionMessage(content='angry', role='assistant', function_call=None, tool_calls=None) angry
ChatCompletionMessage(content='relaxed', role='assistant', function_call=None, tool_calls=None) relaxed
ChatCompletionMessage(content='happy', role='assistant', function_call=None, tool_calls=None) happy
ChatCompletionMessage(content='happy', role='assistant', function_call=None, tool_calls=None) happy
ChatCompletionMessage(content='angry', role='assistant', function_call=None, tool_calls=None) angry
ChatCompletionMessage(content='relaxed', role='assistant', function_call=None, tool_calls=None

In [36]:
# For each element in the array, check if output[i][0].content and output[i][1] are same to get accuracy

count = 0
for i in range(len(output)):
    # print(output[i][0].content, output[i][1])
    if output[i][0].content == output[i][1]:
        count += 1
    # IF not same, print both
    # else:
        # print(output[i][0].content, output[i][1])
print(count/len(output))

# Calculate F1 score
from sklearn.metrics import f1_score
y_true = []
y_pred = []
for i in range(len(output)):
    y_true.append(output[i][1])
    y_pred.append(output[i][0].content)
f1_score(y_true, y_pred, average='weighted')

# Calculate precision and recall
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
precision_score(y_true, y_pred, average='weighted', zero_division=1)
recall_score(y_true, y_pred, average='weighted', zero_division=1)
print(precision_score(y_true, y_pred, average='weighted',zero_division=1), recall_score(y_true, y_pred, average='weighted',zero_division=0))


0.6438356164383562
0.6646508147401536 0.6438356164383562


In [37]:
y_true

['relaxed',
 'happy',
 'angry',
 'sad',
 'angry',
 'relaxed',
 'happy',
 'happy',
 'angry',
 'angry',
 'angry',
 'angry',
 'sad',
 'happy',
 'happy',
 'happy',
 'angry',
 'sad',
 'happy',
 'relaxed',
 'angry',
 'happy',
 'relaxed',
 'happy',
 'sad',
 'sad',
 'relaxed',
 'happy',
 'happy',
 'relaxed',
 'happy',
 'happy',
 'happy',
 'relaxed',
 'angry',
 'relaxed',
 'relaxed',
 'relaxed',
 'sad',
 'angry',
 'sad',
 'angry',
 'happy',
 'angry',
 'relaxed',
 'relaxed',
 'relaxed',
 'angry',
 'happy',
 'relaxed',
 'relaxed',
 'angry',
 'sad',
 'happy',
 'relaxed',
 'happy',
 'relaxed',
 'angry',
 'sad',
 'angry',
 'happy',
 'sad',
 'sad',
 'happy',
 'happy',
 'angry',
 'happy',
 'happy',
 'sad',
 'angry',
 'relaxed',
 'happy',
 'angry']

In [38]:
# Get confusion matrix
print(len(y_true), len(y_pred))
from sklearn.metrics import confusion_matrix
confusion_matrix(y_true, y_pred, labels=['happy', 'sad', 'angry', 'relaxed'])

73 73


array([[14,  1,  2,  7],
       [ 0,  7,  3,  2],
       [ 0,  3, 14,  2],
       [ 4,  2,  0, 12]])

In [39]:
# Get classification report
from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred, labels=['happy', 'sad', 'angry', 'relaxed']))

              precision    recall  f1-score   support

       happy       0.78      0.58      0.67        24
         sad       0.54      0.58      0.56        12
       angry       0.74      0.74      0.74        19
     relaxed       0.52      0.67      0.59        18

    accuracy                           0.64        73
   macro avg       0.64      0.64      0.64        73
weighted avg       0.66      0.64      0.65        73



In [130]:
print(answer.choices[0].message)

ChatCompletionMessage(content='sad', role='assistant', function_call=None, tool_calls=None)


In [133]:
test[0]['messages'][2]['content']

'sad'

In [ ]:
response = client.chat.completions.create(
  model="ft:gpt-3.5-turbo-1106:personal::8OVFs1AY",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello!"}
  ]
)
print(completion.choices[0].message)